# Text2Speech + Face Sync + Lip Sync

In [ ]:
!git clone https://github.com/BrightBlueCheese/ToyFaceAnimation.git

Cloning into 'ToyFaceAnimation'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 126 (delta 16), reused 31 (delta 12), pack-reused 91
Receiving objects: 100% (126/126), 42.65 MiB | 31.21 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Error downloading object: checkpoints.zip (e714f85): Smudge error: Error downloading checkpoints.zip (e714f8562f1701fd07d5d6adf83635e5f4ac5d110cf9a7237f03c1938edfa3e4): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/ToyFaceAnimation/.git/lfs/logs/20231026T051627.738154737.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: checkpoints.zip: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'



In [ ]:
from IPython.display import clear_output

In [ ]:
!sudo apt-get install espeak-ng

clear_output()
print("\nDone")


Done


In [ ]:
!pip install TTS face_recognition pydub

clear_output()
print("\nDone")


Done


In [ ]:
from moviepy.editor import VideoFileClip
from ToyFaceAnimation.rough_packages import frame_definer, source_video_cropper, final_video_maker
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pydub.utils import mediainfo
from IPython.display import HTML, Audio
from TTS.api import TTS

import face_recognition
import os



In [ ]:
pwd

'/content'

#### Path Definition

In [ ]:
# cur_dir = os.getcwd()
cur_dir = os.getcwd() + "/ToyFaceAnimation"

rough_path = f'{cur_dir}/dataset'
os.makedirs(rough_path, exist_ok=True)
rough_path2 = f'{cur_dir}/output'
os.makedirs(rough_path2, exist_ok=True)

source_audio_path = f"{rough_path2}/tts_audio.wav"

In [ ]:
os.chdir(f'{cur_dir}/Wav2Lip')
!pip install -r requirements.txt
!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

clear_output()
os.chdir(f'{cur_dir}')

In [ ]:
cur_dir

'/content/ToyFaceAnimation'

#### Upload data - If you want to use own data
- have to change 'video_name' and 'source_image_path' in this case.

In [ ]:
# Upload data
os.chdir(rough_path)
from google.colab import files
uploaded = files.upload()
os.chdir(cur_dir)

Saving drlang3.png to drlang3.png


## Text To Speech

In [ ]:
os.chdir(cur_dir)
!pwd

/content/ToyFaceAnimation


In [ ]:
# from IPython.display import Audio
# from TTS.api import TTS

In [ ]:
tts = TTS(model_name="tts_models/en/vctk/vits", progress_bar=True, gpu=False)

 > Downloading model to /root/.local/share/tts/tts_models--en--vctk--vits


100%|██████████| 148M/148M [00:06<00:00, 22.7MiB/s]


 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > initialization of speaker-embedding layers.


  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")



### Enter the text for AI speech

In [ ]:
target_text = """
Hello, Everyone!
I have something to announce.
You won't have any class tomorrow. Since tomorrow is Saturday.

Oh. One more thing!
No class on the day after two! Because it will be on Sunday.

Congratulations!

I will go to bed late and wake up late for sure.

Just in case, I am Dr. Lang. But I am not.

Have a blessed weekend!
"""

In [ ]:
# 0 if male else 1
target_gender = 0

if target_gender == 0:
  target_speaker = 'p317'
elif target_gender == 1:
  target_speaker = 'p273'
else:
  print(f"You entered {target_gender}. Please enter valid input. (0 for male, 1 for female)")

In [ ]:
tts.tts_to_file(text=target_text, file_path=source_audio_path, speaker=target_speaker)
audio_widget = Audio(source_audio_path)
audio_widget

 > Text splitted to sentences.
['Hello, Everyone!', 'I have something to announce.', "You won't have any class tomorrow.", 'Since tomorrow is Saturday.', 'Oh.', 'One more thing!', 'No class on the day after two!', 'Because it will be on Sunday.', 'Congratulations!', 'I will go to bed late and wake up late for sure.', 'Just in case, I am Dr. Lang.', 'But I am not.', 'Have a blessed weekend!']
 > Processing time: 23.564023733139038
 > Real-time factor: 0.9629109031054778


### Source Video Crop

In [ ]:
video_name = 'youngmin02.mp4'
video_path = f'{rough_path}/{video_name}'

source_video_path = f"{rough_path2}/cropped_video.mp4"

# def source_video_cropper(video_input:str, audio_input:str='./output/tts_audio.wav',
#                         video_output:str='./output/cropped_video.mp4', video_fps:int=30,
#                         video_res:int=256):
source_video_cropper(video_path, audio_input=source_audio_path, video_output=source_video_path)

Moviepy - Building video /content/ToyFaceAnimation/output/cropped_video.mp4.
MoviePy - Writing audio in cropped_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/ToyFaceAnimation/output/cropped_video.mp4



Moviepy - Done !
Moviepy - video ready /content/ToyFaceAnimation/output/cropped_video.mp4


## Face Sync

In [ ]:

# !mkdir checkpoints
os.makedirs(f"{cur_dir}/checkpoints", exist_ok=True)
# !pip3 install wldhx.yadisk-direct

  # My code is optimized for vox
if not os.path.exists(f"{cur_dir}/checkpoints/vox.pth.tar"):
  command = f"wget -c https://drive.google.com/file/d/1-CKOjv_y_TzNe-dwQsjjeVxJUuyBAb5X/view?usp=drive_link -o checkpoints/vox.pth.tar"
  # command = f"wget -c https://disk.yandex.com/d/bWopgbGj1ZUV1w/vox.pth.tar -o {cur_dir}/checkpoints/vox.pth.tar"
  !{command}

# !curl -L $(yadisk-direct https://disk.yandex.com/d/i08z-kCuDGLuYA) -o checkpoints/vox.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/vk5dirE6KNvEXQ) -o checkpoints/taichi.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/IVtro0k2MVHSvQ) -o checkpoints/mgif.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/B3ipFzpmkB1HIA) -o checkpoints/ted.pth.tar

In [ ]:
import torch

# edit the config
device = torch.device('cuda:0')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = f'{rough_path}/KStit_320.jpg'
# source_image_path = './assets/KStit_320.jpg'
driving_video_path = source_video_path
output_video_path = f'{rough_path2}/face_sync.mp4'
config_path = f'{cur_dir}/FS/config/vox-256.yaml'
checkpoint_path = f'{cur_dir}/checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

# if find_best_frame:
#   !pip install face_alignment

In [ ]:
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from skimage.transform import resize
# from IPython.display import HTML
import warnings

warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

In [ ]:
os.chdir(f'{cur_dir}/FS')
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

os.chdir(cur_dir)

## LipSync

In [ ]:
if not os.path.exists(f"{cur_dir}/checkpoints/wav2lip_gan.pth"):
  command = f"wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '{cur_dir}/checkpoints/wav2lip_gan.pth'"
  !{command}
  command = "pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl"
  !{command}

#download pretrained model for face detection
if not os.path.exists(f"{cur_dir}/Wav2Lip/face_detection/detection/sfd/s3fd.pth"):
  command = f'wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "{cur_dir}/Wav2Lip/face_detection/detection/sfd/s3fd.pth"'
  !{command}

from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

clear_output()
os.chdir(f'{cur_dir}')
print("\nDone")



Done


In [ ]:
final_output_video_path = f"{rough_path2}/face_sync_final.mp4"

# def final_video_maker(video_input:str='./output/face_sync.mp4', audio_input:str='./output/tts_audio.wav',
#                     video_output:str='./output/face_sync_final.mp4'):
final_video_maker(video_input=output_video_path, audio_input=source_audio_path, video_output=final_output_video_path)

Moviepy - Building video /content/ToyFaceAnimation/output/face_sync_final.mp4.
Moviepy - Writing video /content/ToyFaceAnimation/output/face_sync_final.mp4



Moviepy - Done !
Moviepy - video ready /content/ToyFaceAnimation/output/face_sync_final.mp4


In [ ]:
os.chdir(f'{cur_dir}/Wav2Lip')
command = f'python {cur_dir}/Wav2Lip/inference.py --checkpoint_path {cur_dir}/checkpoints/wav2lip_gan.pth --face "{final_output_video_path}" --audio "{source_audio_path}" --outfile "{rough_path2}/final_outcome.mp4"'
!{command}
os.chdir(f'{cur_dir}')

clear_output()
print("\nDone")


Done


In [ ]:

mp4 = open(f"{rough_path2}/final_outcome.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

## Download the outcome mp4

In [ ]:
from google.colab import files
files.download(f"{rough_path2}/final_outcome.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>